# **Import Modules and Libraries**

## *python*

In [ ]:
import os,sys
from transformers import (AutoTokenizer, 
                            AutoModelForTokenClassification)
import shap

## *custom*

In [ ]:
#import customised modules

#define path
sys.path.append(os.path.abspath(os.path.join('..')))

##import relevant modules
try:
    from scripts._03_1_fine_tuning_afroxlmr import afroxlmr_Amharic_NER_fine_tuner
    from scripts._03_2_fine_tuning_bert import bert_Amharic_NER_fine_tuner
    from scripts._04_1_model_interpretability import predict_with_logits
    from scripts._04_2_lime_explainer import LimeNERExplainer
    from scripts._04_3_shap_explainer import ShapNERExplainer
    
except ImportError:
    print('Could not import module from `scripts/`.')

# **afroxlmr-large-ner-masakhaner**

In [ ]:
# Define inputs
conll_path = os.path.join(os.path.dirname(os.getcwd()), 'data', 'ner_amharic_conll.txt')
output_dir = os.path.join(os.path.dirname(os.getcwd()), 'models', 'afroxlmr')

trainer = afroxlmr_Amharic_NER_fine_tuner(conll_path, output_dir = output_dir)
trainer.model = AutoModelForTokenClassification.from_pretrained(output_dir)
trainer.tokenizer = AutoTokenizer.from_pretrained(output_dir)

trainer.label2id = trainer.model.config.label2id
trainer.id2label  =  trainer.model.config.id2label

In [ ]:
tokens = ["ምርጥ","የስጋ","መፍጫ", 
        "ባልቻ", "ሆስፒታል", "አጠገብ","ያለው","ህንፃ", "ላይ", "3ተኛ","ፎቅ", "የሱቅ", "ቁጥር", "3", "ውስጥ",
        "በታላቅ", "ቅናሽ","በ3000", "ብር","ብቻ", "ያገኛሉ", "ለበለጠ", "መረጃ", "በ0911223344", "ላይ", "ይደውሉልን"]
output = predict_with_logits(
    model = trainer.model,
    tokenizer = trainer.tokenizer,
    id2label = trainer.id2label,
    tokens = tokens
    )

print("\n* Word-Level Predictions:")
for t, tag in zip(tokens, output["predicted_labels_word_level"]):
    print(f"{t:10} → {tag}")


In [ ]:
explainer = LimeNERExplainer(
    model = trainer.model,
    tokenizer = trainer.tokenizer,
    id2label = trainer.id2label
)

text = "ምርጥ የስጋ መፍጫ ባልቻ ሆስፒታል አጠገብ ያለው ህንፃ ላይ 3ተኛ ፎቅ የሱቅ ቁጥር 3 ውስጥ" \
        " በታላቅ ቅናሽ በ3000 ብር ብቻ ያገኛሉ ለበለጠ መረጃ በ0911223344 ላይ ይደውሉልን"
explanation = explainer.explain(text, predictor_fn = predict_with_logits)
explanation.show_in_notebook()
explainer.visualise_explanation(text, explanation)

In [ ]:
shap_explainer = ShapNERExplainer(
    model = trainer.model, 
    tokenizer = trainer.tokenizer, 
    id2label = trainer.id2label
    )

text = "ምርጥ የስጋ መፍጫ ባልቻ ሆስፒታል አጠገብ ያለው ህንፃ ላይ 3ተኛ ፎቅ የሱቅ ቁጥር 3 ውስጥ በታላቅ ቅናሽ በ3000 ብር ብቻ ያገኛሉ"
shap_values = shap_explainer.explain(text)
shap_explainer.visualise_shap_explanation(text, shap_values)

shap_explainer.visualise_shap_explanation_thre(text, shap_values, threshold = 0.02)


# Interactive visualization (works in notebooks)
shap.plots.text(shap_values[0])


# **bert-tiny-amharic**

In [ ]:
# Define inputs
conll_path = os.path.join(os.path.dirname(os.getcwd()), 'data', 'ner_amharic_conll.txt')
output_dir = os.path.join(os.path.dirname(os.getcwd()), 'models', 'bert')

trainer = bert_Amharic_NER_fine_tuner(conll_path, output_dir=output_dir)
trainer.model = AutoModelForTokenClassification.from_pretrained(output_dir)
trainer.tokenizer = AutoTokenizer.from_pretrained(output_dir)

trainer.label2id = trainer.model.config.label2id
trainer.id2label = trainer.model.config.id2label

In [ ]:
tokens = ["ምርጥ","የስጋ","መፍጫ", 
        "ባልቻ", "ሆስፒታል", "አጠገብ","ያለው","ህንፃ", "ላይ", "3ተኛ","ፎቅ", "የሱቅ", "ቁጥር", "3", "ውስጥ",
        "በታላቅ", "ቅናሽ","በ3000", "ብር","ብቻ", "ያገኛሉ", "ለበለጠ", "መረጃ", "በ0911223344", "ላይ", "ይደውሉልን"]
output = predict_with_logits(
    model = trainer.model,
    tokenizer = trainer.tokenizer,
    id2label = trainer.id2label,
    tokens = tokens
    )

print("\n* Word-Level Predictions:")
for t, tag in zip(tokens, output["predicted_labels_word_level"]):
    print(f"{t:10} → {tag}")


In [ ]:
explainer = LimeNERExplainer(
    model = trainer.model,
    tokenizer = trainer.tokenizer,
    id2label = trainer.id2label
    )

text = "ምርጥ የስጋ መፍጫ ባልቻ ሆስፒታል አጠገብ ያለው ህንፃ ላይ 3ተኛ ፎቅ የሱቅ ቁጥር 3 ውስጥ በታላቅ ቅናሽ በ3000 ብር ብቻ ያገኛሉ"
explanation = explainer.explain(text, predictor_fn=predict_with_logits)
explanation.show_in_notebook()
explainer.visualise_explanation(text, explanation)

In [ ]:
shap_explainer = ShapNERExplainer(
    model = trainer.model, 
    tokenizer = trainer.tokenizer, 
    id2label = trainer.id2label
    )

text = "ምርጥ የስጋ መፍጫ ባልቻ ሆስፒታል አጠገብ ያለው ህንፃ ላይ 3ተኛ ፎቅ የሱቅ ቁጥር 3 ውስጥ በታላቅ ቅናሽ በ3000 ብር ብቻ ያገኛሉ"
shap_values = shap_explainer.explain(text)
shap_explainer.visualise_shap_explanation(text, shap_values)

shap_explainer.visualise_shap_explanation_thre(text, shap_values, threshold = 0.02)


# Interactive visualization (works in notebooks)
shap.plots.text(shap_values[0])
